In [ ]:
%reload_ext autoreload
%autoreload 2
from importlib import reload


import sys, os, inspect
import sys
sys.path.append('/Users/emigardiner/VICO/pjams-ionization/pjams/')

from zeusmp_snapshot_reader import read_zeusmp_snapshot
from zeusmp_snapshot_reader import ScaleFactors
from snapshot import snapshot 
from basic_snapshot import basic_snapshot, VICO_loc
import plot as plot
import slices as slices
import projs as projs
import ionfrac as ionfrac
# from ionfrac import VMIN, VMAX
# import flux

import numpy as np

import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from matplotlib import cm
from matplotlib.colors import TwoSlopeNorm

from PIL import Image, ImageDraw, ImageFont
from IPython.display import Image as DisplayImage

In [ ]:
INCLUDE_4000 = True

In [ ]:
nums = np.array([4, 9, 21, 39, 54, 68, 94])
years = np.array(['4,000 yrs', '9,000 yrs', '21,000 yrs', '39,000 yrs', '54,000 yrs', '68,000 yrs', '94,000 yrs'])
masss = np.array([r'1.4 M$_\odot$', r'2 M$_\odot$', r'4 M$_\odot$',
                   r'8 M$_\odot$', r'12 M$_\odot$', r'16 M$_\odot$',
                   r'24 M$_\odot$'])

r_kpc = 1
heights_and_scales = np.load(VICO_loc+'/Data/heights_and_scales.npz')
scales = heights_and_scales['scales'] # AU
heights = heights_and_scales['heights'] # AU  

In [ ]:
# Specific to this file
saveloc = projs.SAVELOC

v_mins = projs.v_mins
vmin_ionfrac, vmax_ionfrac = -3, 0       
v = len(v_mins)-1
vmin_iontemp, vmax_iontemp = 0, 7       
# vmin_intensity, vmax_intensity = -3, 2.8 

vmin_intensity_noratio, vmax_intensity_noratio = -3, 2.8   
vmin_intensity_ratio, vmax_intensity_ratio = -4, 0   

In [ ]:
snaps = np.empty_like(nums, dtype=snapshot)
for ii, num in enumerate(nums):
    snaps[ii] = snapshot(snap=num, name = ('Snap%03d_n' % num), read_zeusmp = False)
    snaps[ii].load_shock_variables()
    snaps[ii].load_fluxes()

Load ionfrac arrays

In [ ]:
for ss in range(0,len(snaps)):
    ionfrac.load_average_ionfrac_array(snaps[ss])
    ionfrac.load_average_ionfrac_array_ratio(snaps[ss], const=False)
    
# Maximum Velocities for Contours
for s in range(len(snaps)): 
    for vv in ionfrac.v_mins:
        ionfrac.max_velocity_proj_array(snaps[s], vv)

Calculate scale factors

In [ ]:
def calculate_scale_factors(Snap):
    scale_factors = Snap.cooling_times/Snap.flow_times
    scale_factors[scale_factors>1.0] = 1.0
    Snap.scale_factors = scale_factors        
    return scale_factors

for snap in snaps:
    print(snap.name)
    calculate_scale_factors(snap)

# Colorbars

In [ ]:
scale=25000

# ionfrac
cname = projs.mass_ionfrac_proj_pcolormesh(snaps[s], years[s], scale, v = v,
                        vmin=vmin_ionfrac, vmax = vmax_ionfrac, 
                        show_xlabels = False, show_ylabels = False,
                        show_contours = False, show_legend = False, 
                        show_cbar = True, saveloc=saveloc+'/cbars', cbar_pad=0.05)
cbimg = plot.save_cbar(cname, savename=saveloc+'/cbar_ionfrac.png', orientation='horizontal', extracrop=0)
display(cbimg)

# iontemp
cname = projs.iontemp_proj_pcolormesh(snaps[s], years[s], scale, 
                        vmin=vmin_iontemp, vmax = vmax_iontemp, 
                        show_xlabels = False, show_ylabels = False, 
                        show_cbar = True, saveloc=saveloc+'/cbars', cbar_pad=0.05)
cbimg = plot.save_cbar(cname, savename=saveloc+'/cbar_iontemp.png', orientation='horizontal', extracrop=0)
display(cbimg)

# intensity flooded
cname = projs.intensity_proj_pcolormesh(snaps[s], years[s], scale, 5,
                                     vmin=vmin_intensity_noratio, 
                                     vmax = vmax_intensity_noratio, 
                        show_xlabels = False, show_ylabels = False, 
                        show_cbar = True, saveloc=saveloc+'/cbars', cbar_pad=0.05)
cbimg = plot.save_cbar(cname, savename=saveloc+'/cbar_intensity_flooded.png', orientation='horizontal', extracrop=0)
display(cbimg)

# intensity with cooling
cname = projs.intensity_proj_const_pcolormesh(snaps[s], years[s], False, 
                                     scale, 5,
                                     vmin=vmin_intensity_ratio, 
                                     vmax = vmax_intensity_ratio, 
                        show_xlabels = False, show_ylabels = False, 
                        show_cbar = True, saveloc=saveloc+'/cbars', cbar_pad=0.05)
cbimg = plot.save_cbar(cname, savename=saveloc+'/cbar_intensity_cooling.png', orientation='horizontal', extracrop=0)
display(cbimg)


# PIL Images

#### 1 PIL Figure: 25000au Flooded

In [ ]:
nrows = len(snaps) if INCLUDE_4000 else len(snaps)-1
ncols = 4
scale = 25000
projfiles= np.empty((nrows, ncols), dtype=object)

r=0
for s in range(len(snaps)-nrows,len(snaps)):
    print(years[s])
    if(s < len(snaps)-1):
        file0, file1, file2, file3 = \
        projs.proj_images(snaps[s], masss[s], scale,
                    saveloc=saveloc, xlabels=False)
    else:      
        file0, file1, file2, file3 = \
        projs.proj_images(snaps[s], masss[s], scale,
                    saveloc=saveloc, xlabels=True)
    projfiles[r,0] = file0
    projfiles[r,1] = file1
    projfiles[r,2] = file2
    projfiles[r,3] = file3
    r+=1

projfiles_25000au_noratio = projfiles

In [ ]:
allimage, details = plot.pil_image(projfiles_25000au_noratio, debug=False)
# display(allimage)
allimage = plot.pil_projs_cbar(allimage, details, saveloc=saveloc, ratio=False )
display(allimage)
name = VICO_loc+'/figures/projs/projs_caseA_25000au.png' if INCLUDE_4000 else VICO_loc+'/figures/projs/projs_flooded_25000au.png'
allimage.save(name)

#### 2 PIL Figure: 4000au Flooded

In [ ]:
nrows = len(snaps) if INCLUDE_4000 else len(snaps)-1
ncols = 4
scale = 4000
projfiles= np.empty((nrows, ncols), dtype=object)

r=0
for s in range(len(snaps)-nrows,len(snaps)):
    print(years[s])
    if(s < len(snaps)-1):
        file0, file1, file2, file3 = \
        projs.proj_images(snaps[s], masss[s], scale,
                    saveloc=saveloc, xlabels=False)
    else:      
        file0, file1, file2, file3 = \
        projs.proj_images(snaps[s], masss[s], scale,
                    saveloc=saveloc, xlabels=True)
    projfiles[r,0] = file0
    projfiles[r,1] = file1
    projfiles[r,2] = file2
    projfiles[r,3] = file3
    r+=1

projfiles_4000au_noratio = projfiles

In [ ]:
allimage, details = plot.pil_image(projfiles_4000au_noratio, debug=False)
# display(allimage)
allimage = plot.pil_projs_cbar(allimage, details, saveloc=saveloc, ratio=False )
display(allimage)
name = VICO_loc+'/figures/projs/projs_caseA_4000au.png' if INCLUDE_4000 else VICO_loc+'/figures/projs/projs_flooded_4000au.png'
allimage.save(name)

#### 3 PIL Figure: 25000au Cooling

In [ ]:
nrows = len(snaps) if INCLUDE_4000 else len(snaps)-1
ncols = 4
scale = 25000
projfiles= np.empty((nrows, ncols), dtype=object)

r=0
for s in range(len(snaps)-nrows,len(snaps)):
    print(years[s])
    if(s < len(snaps)-1):
        file0, file1, file2, file3 = \
        projs.proj_const_images(snaps[s], masss[s], False, scale,
                    saveloc=saveloc, xlabels=False)
    else:      
        file0, file1, file2, file3 = \
        projs.proj_const_images(snaps[s], masss[s], False, scale,
                    saveloc=saveloc, xlabels=True)
    projfiles[r,0] = file0
    projfiles[r,1] = file1
    projfiles[r,2] = file2
    projfiles[r,3] = file3
    r+=1

projfiles_25000au_ratio = projfiles

In [ ]:
allimage, details = plot.pil_image(projfiles_25000au_ratio, debug=False)
# display(allimage)
allimage = plot.pil_projs_cbar(allimage, details, saveloc=saveloc, ratio=True )
display(allimage)
name = VICO_loc+'/figures/projs/projs_caseB_25000au.png' if INCLUDE_4000 else VICO_loc+'/figures/projs/projs_cooling_25000au.png'
allimage.save(name)

#### 4 PIL Figure: 4000au Cooling

In [ ]:
nrows = len(snaps) if INCLUDE_4000 else len(snaps)-1
ncols = 4
scale = 4000
projfiles= np.empty((nrows, ncols), dtype=object)

r=0
for s in range(len(snaps)-nrows,len(snaps)):
    print(years[s])
    if(s < len(snaps)-1):
        file0, file1, file2, file3 = \
        projs.proj_const_images(snaps[s], masss[s], False, scale,
                    saveloc=saveloc, xlabels=False)
    else:      
        file0, file1, file2, file3 = \
        projs.proj_const_images(snaps[s], masss[s], False, scale,
                    saveloc=saveloc, xlabels=True)
    projfiles[r,0] = file0
    projfiles[r,1] = file1
    projfiles[r,2] = file2
    projfiles[r,3] = file3
    r+=1

projfiles_4000au_ratio = projfiles

In [ ]:
allimage, details = plot.pil_image(projfiles_4000au_ratio, debug=False)
# display(allimage)
allimage = plot.pil_projs_cbar(allimage, details, saveloc=saveloc, ratio=True )
display(allimage)
name = VICO_loc+'/figures/projs/projs_caseB_4000au.png' if INCLUDE_4000 else VICO_loc+'/figures/projs/projs_cooling_4000au.png'
allimage.save(name)

In [ ]:
snap = snaps[3]
print(snap.name)
print(np.max(np.where(snap.x1<2000)))
print(snap.x1[65])
print(np.min(np.where(snap.x1>5000)))


In [ ]:
Snap=snap
TprojA = np.zeros((len(Snap.x1), len(Snap.x2)))
for i in range(len(Snap.x1)):
    for j in range(len(Snap.x2)):
        TprojA[i,j] = (np.sum(Snap.temperatures[i,j,:] * Snap.ion_fractions[i,j,:] * Snap.q['d'][i,j,:] * Snap.del3) 
                        / np.sum(Snap.ion_fractions[i,j,:] * Snap.q['d'][i,j,:] * Snap.del3))


In [ ]:
print(TprojA.shape)

In [ ]:
print(np.min(np.where(snap.x2)))

In [ ]:
print(np.log10(np.nanmax(TprojA[65:100,:])))